In [40]:
# 导包
import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn 
import torch.optim as optim

# data -准备数据

In [41]:
# 加载MNIST训练数据集
train_data = datasets.MNIST(
    root="../data/mnist",               # 指定数据集下载和存储的根目录
    train=True,                         # train=True表示加载训练集
    transform=transforms.ToTensor(),    # 将图像数据转换为PyTorch张量，并将像素值归一化到[0,1]
    download=True                       # 如果root目录下数据集不存在，则自动下载,存在就不会再次下载了
)
# 加载MNIST测试数据集
test_data = datasets.MNIST(root="../data/mnist",train=False,transform=transforms.ToTensor(),download=True)

In [42]:
# 定义批次大小(batch_size)
# - 将整个数据集分成多个小批次进行训练，而不是一次性使用所有数据
# - 好处：1.减少内存占用 2.加快训练速度 3.引入随机性，有助于模型泛化
# - 如果batch_size=1为随机梯度下降，如果等于数据集大小为全批量梯度下降,这里就是随机小批量梯度下降
batch_size = 100

# 创建训练数据加载器 shuffle=True: 数据打乱,每轮训练开始时,会将数据进行随机打乱,然后依次取出batch_size个样本
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)


# 创建测试数据加载器  shuffle=False因为测试时不需要随机性 , 我们希望得到确定的、可复现的结果
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

# net - 定义网络结构

In [43]:
# 定义 MLP 网络  继承nn.Module
class MLP(nn.Module):
    
    # 初始化方法
    # input_size输入数据的维度    
    # hidden_size 隐藏层的大小
    # num_classes 输出分类的数量
    def __init__(self, input_size, hidden_size, num_classes):
        # 调用父类的初始化方法
        super().__init__() 
        # 这里只是定义,顺序不影响,实际计算时会按照forward函数的顺序执行
        # 定义第1个全连接层  
        self.fc1 = nn.Linear(input_size, hidden_size)
        # 定义激活函数
        self.relu = nn.ReLU()
        # 定义第2个全连接层
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # 定义第3个全连接层
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    # 定义forward函数
    # x 输入的数据
    def forward(self, x):
        # 网络结构: x->(fc1->relu)->(fc2->relu)->fc3->out
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        # 将第二层结果传递给fc3: 第二个隐藏层 -> 输出层
        # 输出层不使用ReLU的原因:
        # 1. 这是一个分类问题,最后一层需要得到每个类别的得分(logits)
        # 2. 这些得分会被后续的CrossEntropyLoss处理,其内部包含了softmax激活函数
        # 3. 如果在这里加ReLU会限制输出为非负数,影响模型的表达能力
        out = self.fc3(out)
        # 返回结果
        return out
    

# 初始化MLP 
# - input_size为28*28是因为MNIST数据集的每张图片是28x28像素
# - hidden_size可以根据需要调整，通常选择较大的值以增加模型的表达能力
# - num_classes为10是因为MNIST数据集有10个数字类别（0-9）   
model = MLP(input_size=28*28, hidden_size=512, num_classes=10)

# loss -定义损失函数

In [44]:
# 交叉熵损失函数:适用于多分类问题
criterion = nn.CrossEntropyLoss()

# optim - 定义优化器

In [45]:
# 创建优化器
# - 使用Adam优化器，它结合了动量(Momentum)和自适应学习率(RMSprop)的优点,是最常用的优化器之一
# - model.parameters()获取模型中所有需要优化的参数
# - lr=learning_rate设置学习率 : 控制每次参数更新的步长，太大可能导致震荡，太小会导致收敛慢
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training - 训练模型

In [46]:
# 训练网络
# 定义训练轮数(epochs)：将整个数据集完整训练10次
# 为什么需要多轮训练:
# 1. 单轮训练可能不足以让模型充分学习数据中的特征和规律
# 2. 每轮训练会以不同的随机顺序遍历数据，增加模型见到不同数据组合的机会
# 3. 多轮训练让模型参数能够逐步优化，达到更好的收敛效果
# 4. 通过观察多轮训练的loss变化，可以判断模型是否过拟合或欠拟合
num_epochs = 10  # 定义训练轮数：将整个数据集完整训练10次

# 外层循环：遍历每个epoch
for epoch in range(num_epochs):
    # 内层循环：遍历训练数据加载器中的每个批次
    # images: 一个批次的图像数据
    # labels: 对应的标签
    for i, (images, labels) in enumerate(train_loader):
        # 预处理：将图像数据展平成一维向量 :  -1表示自动计算这个维度，28*28是MNIST图像的像素数
        images = images.reshape(-1, 28 * 28)
        
        # 前向传播：将输入数据送入模型，得到预测结果
        outputs = model(images)
        
        # 计算损失：使用交叉熵损失函数计算预测值和真实标签之间的差异
        loss = criterion(outputs, labels)
        
        # 反向传播前先清零梯度 : 防止梯度累加，确保每次更新参数时只考虑当前批次的梯度
        # 清零梯度是因为PyTorch默认会累积梯度
        # 每次调用loss.backward()时，梯度会被累加到之前的梯度上
        # 这会导致梯度计算错误，因此在每次反向传播前需要清零梯度
        optimizer.zero_grad()
        
        # 反向传播：计算损失函数对每个参数的梯度
        loss.backward()
        
        # 更新参数：使用优化器根据计算得到的梯度更新模型参数
        optimizer.step()
        
        # 每训练100个批次打印一次训练信息
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], ' # 当前轮数/总轮数
                  f'Step [{i+1}/{len(train_loader)}], ' # 当前批次/总批次数
                  f'Loss: {loss.item():.4f}') # 当前批次的损失值，保留4位小数
# 每个批次100张图像,一共有6w张图像,所以每轮需要600批次,才能训练完
# 可以看到损失函数值逐渐在减小(也存在震荡,但是整体是减小的)，说明模型在学习数据的规律

Epoch [1/10], Step [100/600], Loss: 0.2888
Epoch [1/10], Step [200/600], Loss: 0.1070
Epoch [1/10], Step [300/600], Loss: 0.2887
Epoch [1/10], Step [400/600], Loss: 0.0895
Epoch [1/10], Step [500/600], Loss: 0.1063
Epoch [1/10], Step [600/600], Loss: 0.2541
Epoch [2/10], Step [100/600], Loss: 0.1433
Epoch [2/10], Step [200/600], Loss: 0.0451
Epoch [2/10], Step [300/600], Loss: 0.0750
Epoch [2/10], Step [400/600], Loss: 0.1178
Epoch [2/10], Step [500/600], Loss: 0.1866
Epoch [2/10], Step [600/600], Loss: 0.0323
Epoch [3/10], Step [100/600], Loss: 0.0524
Epoch [3/10], Step [200/600], Loss: 0.1047
Epoch [3/10], Step [300/600], Loss: 0.1513
Epoch [3/10], Step [400/600], Loss: 0.0445
Epoch [3/10], Step [500/600], Loss: 0.0246
Epoch [3/10], Step [600/600], Loss: 0.0830
Epoch [4/10], Step [100/600], Loss: 0.0213
Epoch [4/10], Step [200/600], Loss: 0.0144
Epoch [4/10], Step [300/600], Loss: 0.0285
Epoch [4/10], Step [400/600], Loss: 0.0438
Epoch [4/10], Step [500/600], Loss: 0.0154
Epoch [4/10

# test - 模型评估

In [ ]:
# 测试网络 : 在测试阶段,不需要计算梯度,使用torch.no_grad()可以减少内存使用和加快计算速度
with torch.no_grad():  
    correct = 0  
    total = 0    
    # 从 test_loader中循环读取测试数据(图像和对应标签)
    for images, labels in test_loader:
        # 预处理：将图像数据(原来是28*28的二维)展平成一维向量 
        # -1表示自动计算这个维度，28*28是MNIST图像的像素数
        images = images.reshape(-1, 28 * 28)  
        # 将数据送给网络进行前向传播,得到预测结果
        outputs = model(images) 
        # 取出最大值对应的索引  即预测值
        # 获取预测结果
        # torch.max(outputs.data, 1)返回两个值:
        # 1. 每一行中最大的值 (在这里不需要,用_占位)
        # 2. 最大值对应的索引 (即模型预测的类别)
        # dim=1表示在第1维度上取最大值(即每行)
        _, predicted = torch.max(outputs.data, dim=1)  
        # 累加当前批次的样本总数
        total += labels.size(0) 
        # 预测值与labels值比对 获取预测正确的数量
        correct += (predicted == labels).sum().item()  
    # 打印最终的准确率
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')  

Accuracy of the network on the 10000 test images: 98.16 %


# save -保存训练好的模型

In [48]:
torch.save(model,"mnist_mlp_model.pkl")